In [405]:
# IMPORT STATEMENTS

#Import Python packages
import pandas as pd
import json
import matplotlib.pyplot as plt
import json

from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

import ast

# to split training and testing data
from sklearn.model_selection import train_test_split

# the scaler - for min-max scaling
from sklearn.preprocessing import MinMaxScaler

# import XGBRegressor from xgboost package
from xgboost import XGBRegressor

# Import Snowflake modules
from snowflake.snowpark import Session

In [406]:
# Get account credentials from a json file
with open("data_scientist_auth.json") as f:
    data = json.load(f)
    username = data["username"]
    password = data["password"]
    account = data["account"]

# Specify connection parameters
connection_parameters = {
    "account": account,
    "user": username,
    "password": password,
    "role": "TASTY_BI",
    "warehouse": "TASTY_BI_WH",
    "database": "frostbyte_tasty_bytes",
    "schema": "analytics",
}

# Create Snowpark session
session = Session.builder.configs(connection_parameters).create()

In [407]:
# retrieve order details usa table from snowflake
order_data_from_snowflake = session.table("frostbyte_tasty_bytes.analytics.ORDER_DETAILS_USA_MATCHED")

In [408]:
# convert order_data_from_snowflake to pandas dataframe
order_df = order_data_from_snowflake.to_pandas()

In [409]:
# retrieve menu data from snowflake
menu_data_from_snowflake = session.table("frostbyte_tasty_bytes.raw_pos.MENU")

In [410]:
# convert menu_data_from_snowflake to pandas dataframe
menu_df = menu_data_from_snowflake.to_pandas()

In [411]:
# preview order table
order_df.head()

,ORDER_ID,CUSTOMER_ID,TRUCK_ID,LOCATION_ID,SHIFT_ID,SHIFT_START_TIME,SHIFT_END_TIME,ORDER_CHANNEL,ORDER_TS,SERVED_TS,...,E_MAIL,PHONE_NUMBER,ORDER_DETAIL_ID,MENU_ITEM_ID,DISCOUNT_ID,LINE_NUMBER,QUANTITY,UNIT_PRICE,PRICE,ORDER_ITEM_DISCOUNT_AMOUNT
0,447758081,101191,61,15475.0,200524445,15:30:00,22:30:00,None,2022-09-22 18:13:43,None,...,Zain.Sullivan@hotmail.com,659-899-8290,872457141,19,None,0,1,3.0,3.0,None
1,447758081,101191,61,15475.0,200524445,15:30:00,22:30:00,None,2022-09-22 18:13:43,None,...,Zain.Sullivan@hotmail.com,659-899-8290,872457142,11,None,1,1,6.0,6.0,None
2,447758081,101191,61,15475.0,200524445,15:30:00,22:30:00,None,2022-09-22 18:13:43,None,...,Zain.Sullivan@hotmail.com,659-899-8290,872457143,15,None,2,1,3.0,3.0,None
3,447759739,195383,62,2588.0,200524447,15:30:00,22:30:00,None,2022-09-22 19:46:15,None,...,Aryana.Dennis@hotmail.com,437-446-0786,872462265,24,None,0,1,2.0,2.0,None
4,447759739,195383,62,2588.0,200524447,15:30:00,22:30:00,None,2022-09-22 19:46:15,None,...,Aryana.Dennis@hotmail.com,437-446-0786,872462266,27,None,1,1,6.0,6.0,None


In [412]:
# preview menu table
menu_df.head()

,MENU_ID,MENU_TYPE_ID,MENU_TYPE,TRUCK_BRAND_NAME,MENU_ITEM_ID,MENU_ITEM_NAME,ITEM_CATEGORY,ITEM_SUBCATEGORY,COST_OF_GOODS_USD,SALE_PRICE_USD,MENU_ITEM_HEALTH_METRICS_OBJ
0,10001,1,Ice Cream,Freezing Point,10,Lemonade,Beverage,Cold Option,0.65,3.5,"{\n ""menu_item_health_metrics"": [\n {\n ..."
1,10002,1,Ice Cream,Freezing Point,11,Sugar Cone,Dessert,Cold Option,2.50,6.0,"{\n ""menu_item_health_metrics"": [\n {\n ..."
2,10003,1,Ice Cream,Freezing Point,12,Waffle Cone,Dessert,Cold Option,2.50,6.0,"{\n ""menu_item_health_metrics"": [\n {\n ..."
3,10004,1,Ice Cream,Freezing Point,13,Two Scoop Bowl,Dessert,Cold Option,3.00,7.0,"{\n ""menu_item_health_metrics"": [\n {\n ..."
4,10005,1,Ice Cream,Freezing Point,14,Bottled Water,Beverage,Cold Option,0.50,2.0,"{\n ""menu_item_health_metrics"": [\n {\n ..."


In [413]:
# Convert the string JSON data to a nested dictionary
menu_df['MENU_ITEM_HEALTH_METRICS_OBJ'] = menu_df['MENU_ITEM_HEALTH_METRICS_OBJ'].apply(ast.literal_eval)

# Use json_normalize to flatten the nested JSON data
menu_item_metrics = pd.json_normalize(menu_df['MENU_ITEM_HEALTH_METRICS_OBJ'], record_path='menu_item_health_metrics')

# Rename the columns
menu_item_metrics = menu_item_metrics.rename(columns={
    'is_dairy_free_flag': 'DAIRY_FREE',
    'is_gluten_free_flag': 'GLUTEN_FREE',
    'is_healthy_flag': 'HEALTHY',
    'is_nut_free_flag': 'NUT_FREE'
})

# Replace 'Y' with 'Yes' and 'N' with 'No' in the DataFrame
menu_item_metrics = menu_item_metrics.replace({'Y': 1, 'N': 0})

# Concatenate the flattened DataFrame with the original DataFrame
menu_df = pd.concat([menu_df, menu_item_metrics], axis=1)

# Drop the original 'MENU_ITEM_HEALTH_METRICS_OBJ' and 'ingredients' column 
menu_df = menu_df.drop(columns=['MENU_ITEM_HEALTH_METRICS_OBJ', 'ingredients'])

In [414]:
menu_df.head()

,MENU_ID,MENU_TYPE_ID,MENU_TYPE,TRUCK_BRAND_NAME,MENU_ITEM_ID,MENU_ITEM_NAME,ITEM_CATEGORY,ITEM_SUBCATEGORY,COST_OF_GOODS_USD,SALE_PRICE_USD,DAIRY_FREE,GLUTEN_FREE,HEALTHY,NUT_FREE
0,10001,1,Ice Cream,Freezing Point,10,Lemonade,Beverage,Cold Option,0.65,3.5,1,1,0,1
1,10002,1,Ice Cream,Freezing Point,11,Sugar Cone,Dessert,Cold Option,2.50,6.0,1,1,0,1
2,10003,1,Ice Cream,Freezing Point,12,Waffle Cone,Dessert,Cold Option,2.50,6.0,0,0,0,1
3,10004,1,Ice Cream,Freezing Point,13,Two Scoop Bowl,Dessert,Cold Option,3.00,7.0,0,1,0,1
4,10005,1,Ice Cream,Freezing Point,14,Bottled Water,Beverage,Cold Option,0.50,2.0,1,1,1,1


In [415]:
# Get the total quantity sold for each menu item 
## group by 'MENU_ITEM_ID' and calculate the total quantity sold
total_qty_sold_per_item = order_df.groupby('MENU_ITEM_ID')['QUANTITY'].sum().reset_index()

## rename the 'QUANTITY' column to 'TOTAL_QTY_SOLD'
total_qty_sold_per_item = total_qty_sold_per_item.rename(columns={'QUANTITY': 'TOTAL_QTY_SOLD'})

## merge total_qty_sold_per_item with final_product_df
menu_df = pd.merge(menu_df, total_qty_sold_per_item, on='MENU_ITEM_ID')

In [416]:
menu_df.head()

,MENU_ID,MENU_TYPE_ID,MENU_TYPE,TRUCK_BRAND_NAME,MENU_ITEM_ID,MENU_ITEM_NAME,ITEM_CATEGORY,ITEM_SUBCATEGORY,COST_OF_GOODS_USD,SALE_PRICE_USD,DAIRY_FREE,GLUTEN_FREE,HEALTHY,NUT_FREE,TOTAL_QTY_SOLD
0,10001,1,Ice Cream,Freezing Point,10,Lemonade,Beverage,Cold Option,0.65,3.5,1,1,0,1,607
1,10002,1,Ice Cream,Freezing Point,11,Sugar Cone,Dessert,Cold Option,2.50,6.0,1,1,0,1,1815
2,10003,1,Ice Cream,Freezing Point,12,Waffle Cone,Dessert,Cold Option,2.50,6.0,0,0,0,1,1786
3,10004,1,Ice Cream,Freezing Point,13,Two Scoop Bowl,Dessert,Cold Option,3.00,7.0,0,1,0,1,1682
4,10005,1,Ice Cream,Freezing Point,14,Bottled Water,Beverage,Cold Option,0.50,2.0,1,1,1,1,629


In [417]:
final_df = menu_df.drop(["MENU_ID", "MENU_TYPE_ID", "MENU_ITEM_ID", "MENU_ITEM_NAME", "COST_OF_GOODS_USD"]
                        , axis=1)

In [418]:
final_df.head()

,MENU_TYPE,TRUCK_BRAND_NAME,ITEM_CATEGORY,ITEM_SUBCATEGORY,SALE_PRICE_USD,DAIRY_FREE,GLUTEN_FREE,HEALTHY,NUT_FREE,TOTAL_QTY_SOLD
0,Ice Cream,Freezing Point,Beverage,Cold Option,3.5,1,1,0,1,607
1,Ice Cream,Freezing Point,Dessert,Cold Option,6.0,1,1,0,1,1815
2,Ice Cream,Freezing Point,Dessert,Cold Option,6.0,0,0,0,1,1786
3,Ice Cream,Freezing Point,Dessert,Cold Option,7.0,0,1,0,1,1682
4,Ice Cream,Freezing Point,Beverage,Cold Option,2.0,1,1,1,1,629


In [419]:
X_train, X_test, y_train, y_test = train_test_split(
    final_df.drop('TOTAL_QTY_SOLD', axis=1), final_df['TOTAL_QTY_SOLD'], test_size=0.3, random_state=0)

X_train.shape, X_test.shape

((70, 9), (30, 9))

In [420]:
# let's create the encoder
from feature_engine.encoding import OneHotEncoder

ohe_enc = OneHotEncoder(
    top_categories=None,
    drop_last=True)  # to return k-1, false to return k

In [421]:
# fit the encoder to the train set: it will learn the variables and 
# categories to encode

ohe_enc.fit(X_train)

c:\Users\donsu\.conda\envs\py38_env\lib\site-packages\feature_engine\variable_handling\_variable_type_checks.py:28: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  return is_datetime(pd.to_datetime(column, errors="ignore", utc=True))
c:\Users\donsu\.conda\envs\py38_env\lib\site-packages\feature_engine\variable_handling\_variable_type_checks.py:28: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  return is_datetime(pd.to_datetime(column, errors="ignore", utc=True))
c:\Users\donsu\.conda\envs\py38_env\lib\site-packages\feature_engine\variable_handling\_variable_type_checks.py:28: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consisten

OneHotEncoder(drop_last=True)

In [422]:
X_train = ohe_enc.transform(X_train)
X_test = ohe_enc.transform(X_test)

In [423]:
X_train.head()

,SALE_PRICE_USD,DAIRY_FREE,GLUTEN_FREE,HEALTHY,NUT_FREE,MENU_TYPE_Ethiopian,MENU_TYPE_Gyros,MENU_TYPE_Indian,MENU_TYPE_Hot Dogs,MENU_TYPE_Vegetarian,...,TRUCK_BRAND_NAME_Freezing Point,TRUCK_BRAND_NAME_The Mega Melt,TRUCK_BRAND_NAME_Kitakata Ramen Bar,TRUCK_BRAND_NAME_The Mac Shack,TRUCK_BRAND_NAME_Peking Truck,ITEM_CATEGORY_Main,ITEM_CATEGORY_Beverage,ITEM_CATEGORY_Snack,ITEM_SUBCATEGORY_Warm Option,ITEM_SUBCATEGORY_Cold Option
60,9.0,1,1,1,1,1,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
80,3.0,1,1,0,1,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,1
90,15.0,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
68,3.0,1,1,0,1,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,1
51,3.0,1,1,0,1,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,1


In [424]:
# set up the scaler
scaler = MinMaxScaler()

# fit the scaler to the train set, it will learn the parameters
scaler.fit(X_train)

# transform train and test sets
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [425]:
# let's transform the returned NumPy arrays to dataframes
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)

In [426]:
# Build XGBoost Model
xgb = XGBRegressor(n_estimators = 20, learning_rate = 0.1, use_label_encoder = False, eval_metric='logloss')
# fit the model to the training data
xgb.fit(X_train_scaled, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric='logloss',
             feature_types=None, gamma=None, gpu_id=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.1, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, n_estimators=20, n_jobs=None,
             num_parallel_tree=None, predictor=None, random_state=None, ...)

In [427]:
def model_performance(model):
    print('the training root mean squared error is: ',  np.sqrt(mean_squared_error(model.predict(X_train_scaled), y_train)))
    print('the testing root mean squared error is: ',  np.sqrt(mean_squared_error(model.predict(X_test_scaled), y_test)))

    print()

    #training mse
    train_mse = mean_squared_error(model.predict(X_train_scaled), y_train)
    print('the training mean squared error is: ', train_mse)
    #testing mse
    test_mse = mean_squared_error(model.predict(X_test_scaled), y_test)
    print('the testing mean squared error is: ', test_mse)

    print()

    print('training accuracy is: ', model.score(X_train_scaled, y_train))
    print('testing accuracy is: ', model.score(X_test_scaled, y_test))

# Assuming you have already defined and trained the XGBoost model as shown in the previous code section.

# Call the function to evaluate the XGBoost model
model_performance(xgb)


the training root mean squared error is:  329.1068276729115
the testing root mean squared error is:  310.2555739999605

the training mean squared error is:  108311.30402092748
the testing mean squared error is:  96258.52119804497

training accuracy is:  0.929290223851081
testing accuracy is:  0.9309793320665356


In [428]:
model_performance(xgb)

the training root mean squared error is:  329.1068276729115
the testing root mean squared error is:  310.2555739999605

the training mean squared error is:  108311.30402092748
the testing mean squared error is:  96258.52119804497

training accuracy is:  0.929290223851081
testing accuracy is:  0.9309793320665356
